# Importing Libs

In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

2025-07-13 15:49:23.656341: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data Preprocessing

## Training Set

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(128,128),
    batch_size=32,
    class_mode='binary')

Found 8004 images belonging to 2 classes.


## Test Set

In [3]:
test_datagen = ImageDataGenerator( rescale=1./255)
test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(128,128),
    batch_size=32,
    class_mode='binary')

Found 2004 images belonging to 2 classes.


# Building CNN

## Initializing CNN

In [4]:
cnn = tf.keras.models.Sequential()

2025-07-13 15:49:30.337872: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-07-13 15:49:30.338838: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-07-13 15:49:30.338906: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-07-13 15:49:30.339098: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-13 15:49:30.339321: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Convolutional

In [5]:
cnn.add( tf.keras.layers.Conv2D( filters =32, kernel_size=3, activation ='relu', input_shape= [128,128,3]) )

## Pooling

In [6]:
cnn.add( tf.keras.layers.MaxPool2D( pool_size=2, strides =2) )

## Adding a second Convolutional layer

In [7]:
cnn.add( tf.keras.layers.Conv2D( filters =32, kernel_size=3, activation ='relu') )
cnn.add( tf.keras.layers.MaxPool2D( pool_size=2, strides =2) )

## Flattening

In [8]:
cnn.add( tf.keras.layers.Flatten() )

## Full Connected Layer

In [9]:
cnn.add( tf.keras.layers.Dense( units =128, activation = 'relu') )

## Output Layer

In [10]:
cnn.add( tf.keras.layers.Dense( units=1,activation ='sigmoid' ) )

# Training CNN

## Compiling Cnn

In [11]:
cnn.compile( optimizer ='adam', loss = 'binary_crossentropy', metrics =['accuracy'] )

## Training the cnn on training set and evaluating it on test set

In [12]:
cnn.fit( x=training_set, validation_data= test_set, epochs=25 )

Epoch 1/25


2025-07-13 15:49:46.666131: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2025-07-13 15:49:46.998243: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-07-13 15:49:47.039877: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.


251/251 [==============================] - ETA: 0s - loss: 0.8955 - accuracy: 0.5000

2025-07-13 15:50:16.599039: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2025-07-13 15:50:16.658085: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


251/251 [==============================] - 33s 127ms/step - loss: 0.8955 - accuracy: 0.5000 - val_loss: 0.8419 - val_accuracy: 0.4810
Epoch 2/25
251/251 [==============================] - 30s 119ms/step - loss: 0.8588 - accuracy: 0.5049 - val_loss: 0.6888 - val_accuracy: 0.5509
Epoch 3/25
251/251 [==============================] - 30s 119ms/step - loss: 0.6565 - accuracy: 0.6012 - val_loss: 0.6232 - val_accuracy: 0.6567
Epoch 4/25
251/251 [==============================] - 29s 117ms/step - loss: 0.6106 - accuracy: 0.6694 - val_loss: 0.6036 - val_accuracy: 0.6801
Epoch 5/25
251/251 [==============================] - 31s 122ms/step - loss: 0.5855 - accuracy: 0.6908 - val_loss: 0.5623 - val_accuracy: 0.7216
Epoch 6/25
251/251 [==============================] - 30s 120ms/step - loss: 0.5667 - accuracy: 0.7115 - val_loss: 0.5588 - val_accuracy: 0.7151
Epoch 7/25
251/251 [==============================] - 30s 118ms/step - loss: 0.5463 - accuracy: 0.7264 - val_loss: 0.6734 - val_accuracy: 0.6

# Prediction

In [14]:
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dataset/test_set/cats/Cat.4065.jpg',target_size=(128,128) )
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] >0.5:
    prediction = 'Dog'
else:
    prediction = 'Cat'
    

1/1 [==============================] - 0s 215ms/step


2025-07-13 16:02:37.252583: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [15]:
print(prediction)

Cat


In [16]:

test_image = image.load_img('dataset/test_set/dogs/Dog.4065.jpg',target_size=(128,128) )
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] >0.5:
    prediction = 'Dog'
else:
    prediction = 'Cat'
    

1/1 [==============================] - 0s 25ms/step


In [17]:
print(prediction)

Dog
